In [48]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import regex as re
from nltk.stem import PorterStemmer
from pandarallel import pandarallel
import itertools
import collections
import subprocess
import psutil
import os

stemmer = PorterStemmer()

pandarallel.initialize(progress_bar = False)

# Serialization folder
SERIALIZATION_FOLDER = "pickle/"

# Serialization folder
DF_NAME = "df.pkl"

# Environment
DATA_PATH = 'data/'
EMAIL_DATA = 'Emails.csv'

ASUM_PATH = 'asum'

TOKENS_THLD = 15

FREQ_THLD = 10

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [49]:
df = pd.read_pickle(SERIALIZATION_FOLDER + DF_NAME)

print(f"Length before: {len(df)}")
df = df[df["Tokenized"].apply(lambda x: len(x)) > TOKENS_THLD]
print(f"Length after: {len(df)}")
df.head()

Length before: 6737
Length after: 1752


,SenderPersonId,MetadataDateSent,ExtractedSubject,ExtractedBodyText,DateYear,DateMonth,DateDay,ExtractedBodyTextCleaned,Tokenized
Id,,,,,,,,,
2,NaN,2011-03-03 05:00:00+00:00,NaN,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",2011,3,3,b6\nh: latest how syria is aiding qaddafi and ...,"[h, latest, syria, aiding, qaddafi, sid, hrc, ..."
6,80.0,2012-09-12 04:00:00+00:00,Meet The Right Wing Extremist Behind Anti-Musl...,Pis print.\n-•-...-^\nH < hrod17@clintonernail...,2012,9,12,°russorv@state.gov'\nfrom [meat)\nsent: to: 11...,"[state.gov', meat, sent, subject, meet, right,..."
14,10.0,2011-03-13 05:00:00+00:00,NaN,"Anne-Marie Slaughter\nSunday, March 13, 2011 9...",2011,3,13,"anne-marie slaughter\njacob mills, cheryl d; r...","[anne-marie, slaughter, jacob, mills, cheryl, ..."
15,32.0,2012-09-12 04:00:00+00:00,RE: Not a dry eye in NEA,"_ .....\nFrom Randolph, Lawrence M\nSent: Wedn...",2012,9,12,"_ .....\nfrom randolph, lawrence m\nsent: to: ...","[randolph, lawrence, sent, mills, cheryl, subj..."
16,77.0,2012-09-12 04:00:00+00:00,NaN,I asked to attend your svtc today with Embassy...,2012,9,12,i asked to attend your svtc today with embassy...,"[asked, attend, svtc, today, embassy, tripoli,..."


In [50]:
def tokenize_sentence(sentence, stem=True):
    tokenized = word_tokenize(sentence)
    # Strip tokens
    tokenized = [token.strip() for token in tokenized]
    # Strict regex rule
    tokenized = [token for token in tokenized if re.match('\w+', token)]
    # Remove punctuation
    tokenized = [token for token in tokenized if token not in string.punctuation]
    # Remove stopwords
    stop = stopwords.words('english') + [':', '.', '@'] + ["n't"]
    tokenized = [token for token in tokenized if token not in stop]
    # Remove numbers
    tokenized = [token for token in tokenized if not re.search(r'\d', token)]
    if stem:
        tokenized = [stemmer.stem(token) for token in tokenized]
    return tokenized

def preprocess_asum(body):
    sentences = sent_tokenize(body)
    sentences = [tokenize_sentence(sentence) for sentence in sentences]
    return [s for s in sentences if s != []]

In [51]:
text = df.iloc[4]['ExtractedBodyTextCleaned']
text

"i asked to attend your svtc today with embassy tripoli, because had first met so many of that staff when i went with\nyou from malta to tripoli for the reopening of our embassy.\ntoday's deaths hit me much harder than i would have guessed. i am always proud to serve under you, but never have\nyour words been more meaningful than on today's svtc. every day of your tenure has been extraordinary, but none\nmore so than today. thank you again for your inspirational leadership and example.\nas ever,\nharold\nu.s. department of state\ncase no. f-2015-04841\ndoc no. c05739571\ndate: 05/13/2015\nstate dept. - produced to house select benghazi comm.\nsubject to agreement on sensitive information & redactions. no foia waiver. state-scb0045269"

In [52]:
sent_text = nltk.sent_tokenize(text)
sent_text

['i asked to attend your svtc today with embassy tripoli, because had first met so many of that staff when i went with\nyou from malta to tripoli for the reopening of our embassy.',
 "today's deaths hit me much harder than i would have guessed.",
 "i am always proud to serve under you, but never have\nyour words been more meaningful than on today's svtc.",
 'every day of your tenure has been extraordinary, but none\nmore so than today.',
 'thank you again for your inspirational leadership and example.',
 'as ever,\nharold\nu.s. department of state\ncase no.',
 'f-2015-04841\ndoc no.',
 'c05739571\ndate: 05/13/2015\nstate dept.',
 '- produced to house select benghazi comm.',
 'subject to agreement on sensitive information & redactions.',
 'no foia waiver.',
 'state-scb0045269']

In [53]:
[tokenize_sentence(s) for s in sent_text]

[['ask',
  'attend',
  'svtc',
  'today',
  'embassi',
  'tripoli',
  'first',
  'met',
  'mani',
  'staff',
  'went',
  'malta',
  'tripoli',
  'reopen',
  'embassi'],
 ['today', 'death', 'hit', 'much', 'harder', 'would', 'guess'],
 ['alway', 'proud', 'serv', 'never', 'word', 'meaning', 'today', 'svtc'],
 ['everi', 'day', 'tenur', 'extraordinari', 'none', 'today'],
 ['thank', 'inspir', 'leadership', 'exampl'],
 ['ever', 'harold', 'u.s.', 'depart', 'state', 'case'],
 ['doc'],
 ['date', 'state', 'dept'],
 ['produc', 'hous', 'select', 'benghazi', 'comm'],
 ['subject', 'agreement', 'sensit', 'inform', 'redact'],
 ['foia', 'waiver'],
 []]

In [54]:
df["asum"] = df["ExtractedBodyTextCleaned"].parallel_apply(preprocess_asum)
df.head()

,SenderPersonId,MetadataDateSent,ExtractedSubject,ExtractedBodyText,DateYear,DateMonth,DateDay,ExtractedBodyTextCleaned,Tokenized,asum
Id,,,,,,,,,,
2,NaN,2011-03-03 05:00:00+00:00,NaN,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",2011,3,3,b6\nh: latest how syria is aiding qaddafi and ...,"[h, latest, syria, aiding, qaddafi, sid, hrc, ...","[[h, latest, syria, aid, qaddafi, sid, hrc, me..."
6,80.0,2012-09-12 04:00:00+00:00,Meet The Right Wing Extremist Behind Anti-Musl...,Pis print.\n-•-...-^\nH < hrod17@clintonernail...,2012,9,12,°russorv@state.gov'\nfrom [meat)\nsent: to: 11...,"[state.gov', meat, sent, subject, meet, right,...","[[state.gov', meat, sent, subject, meet, right..."
14,10.0,2011-03-13 05:00:00+00:00,NaN,"Anne-Marie Slaughter\nSunday, March 13, 2011 9...",2011,3,13,"anne-marie slaughter\njacob mills, cheryl d; r...","[anne-marie, slaughter, jacob, mills, cheryl, ...","[[anne-mari, slaughter, jacob, mill, cheryl, r..."
15,32.0,2012-09-12 04:00:00+00:00,RE: Not a dry eye in NEA,"_ .....\nFrom Randolph, Lawrence M\nSent: Wedn...",2012,9,12,"_ .....\nfrom randolph, lawrence m\nsent: to: ...","[randolph, lawrence, sent, mills, cheryl, subj...","[[randolph, lawrenc, sent, mill, cheryl, subje..."
16,77.0,2012-09-12 04:00:00+00:00,NaN,I asked to attend your svtc today with Embassy...,2012,9,12,i asked to attend your svtc today with embassy...,"[asked, attend, svtc, today, embassy, tripoli,...","[[ask, attend, svtc, today, embassi, tripoli, ..."


In [55]:
flatten = list(itertools.chain(*list(itertools.chain(*df["asum"]))))
tokens_freq = collections.Counter(flatten)
print("Describe tokens frequency")
pd.Series(tokens_freq.values()).describe()

Describe tokens frequency


count    20088.000000
mean        13.966697
std         57.815385
min          1.000000
25%          1.000000
50%          2.000000
75%          6.000000
max       2122.000000
dtype: float64

In [56]:
top_tokens = [token for token, occ in tokens_freq.items() if occ > FREQ_THLD]
print(f"After thld on frequency: {len(top_tokens)}")

After thld on frequency: 3419


In [59]:
top_tokens = [token for token in top_tokens if re.match("^[a-z]+$", token)]
top_tokens = set(top_tokens) - {'url', 'http'}
top_tokens = sorted(top_tokens)
top_tokens = {idx: token for idx, token in zip(top_tokens, range(len(top_tokens)))}
top_tokens

{'abandon': 0,
 'abba': 1,
 'abc': 2,
 'abedin': 3,
 'abil': 4,
 'abl': 5,
 'abort': 6,
 'abraham': 7,
 'abroad': 8,
 'absenc': 9,
 'absolut': 10,
 'abu': 11,
 'abus': 12,
 'academ': 13,
 'academi': 14,
 'acceler': 15,
 'accept': 16,
 'access': 17,
 'accommod': 18,
 'accompani': 19,
 'accomplish': 20,
 'accord': 21,
 'accordingli': 22,
 'account': 23,
 'accur': 24,
 'accus': 25,
 'achiev': 26,
 'acknowledg': 27,
 'acorn': 28,
 'across': 29,
 'act': 30,
 'action': 31,
 'activ': 32,
 'activist': 33,
 'actor': 34,
 'actual': 35,
 'ad': 36,
 'adam': 37,
 'adapt': 38,
 'add': 39,
 'addit': 40,
 'address': 41,
 'adequ': 42,
 'adjust': 43,
 'administ': 44,
 'administr': 45,
 'admir': 46,
 'admit': 47,
 'adopt': 48,
 'adult': 49,
 'advanc': 50,
 'advantag': 51,
 'adversari': 52,
 'advertis': 53,
 'advic': 54,
 'advis': 55,
 'advisor': 56,
 'advisori': 57,
 'advoc': 58,
 'advocaci': 59,
 'afb': 60,
 'affair': 61,
 'affect': 62,
 'affili': 63,
 'affirm': 64,
 'afford': 65,
 'afghan': 66,
 'afgha

In [43]:
def resolve_asum(sentences):
    # Resolve indices
    resolved = []
    for sentence in sentences:
        resolved.append([top_tokens.get(token, None) for token in sentence])
    # Remove None values
    no_none = []
    for l in resolved:
        no_none.append([x for x in l if x is not None])
    # Remove empty list
    out_list = []
    for l in no_none:
        out_list.append([x for x in l if x != []])
    # Remove empty body
    return [x for x in out_list if x != []]

In [44]:
final_series = df["asum"].parallel_apply(resolve_asum)
final_series


Id
2       [[1336, 1695, 2945, 85, 2388, 2727, 1428, 1875...
6       [[2677, 2898, 1871, 2567, 3283, 1087, 286, 114...
14      [[2756, 1579, 1906, 487, 3, 2225, 1745, 161, 2...
15      [[2677, 1906, 487, 2898, 1477, 1911], [2507, 2...
16      [[201, 221, 3039, 960, 3092, 1158, 1886, 1820,...
                              ...                        
7934    [[933, 161, 330, 2083, 2321, 1868, 2355, 1477,...
7938    [[748, 1004, 2273, 1691, 2016, 3202, 1719, 140...
7939    [[2792, 467, 2663, 3226, 1371, 830, 1309, 2549...
7942    [[307, 468, 2235, 2672], [2672, 2490, 142, 177...
7944    [[2386, 2241, 1732, 1660, 1380, 782, 1353, 224...
Name: asum, Length: 1752, dtype: object

In [46]:
# Wordlist
with open(os.path.join(ASUM_PATH,'in', 'WordList.txt'), "w+") as f:
    for word in top_tokens.keys():
        f.write(word + "\n")

In [47]:
# Bag of sentences
with open(os.path.join(ASUM_PATH, 'in', 'BagOfSentences.txt'), "w+") as f:
    for senteces in final_series:
        f.write(str(len(senteces)) + "\n")
        for sentence in senteces:
            f.write(' '.join([str(x) for x in sentence]) + '\n')

In [31]:
N_SENTIMENT = '2'
N_TOPICS = '10'
N_ITER = '1000'
N_THREAD = str(psutil.cpu_count())
ALPHA = '0.1'
BETHA = '0.001/0.1/0.1'
G = '1/1'
SCRIPT = '.'.join([ASUM_PATH, 'bin', 'sto2.STO2Core'])
INPUT = os.path.join(ASUM_PATH, 'in')
OUTPUT = os.path.join(ASUM_PATH, 'out')


COMMAND = f'java {SCRIPT} -s {N_SENTIMENT} -t {N_TOPICS} -i {N_ITER} ' \
          f'-th {N_THREAD} -a {ALPHA} -b {BETHA} -g {G} -d {INPUT} -o {OUTPUT}'
COMMAND

'java asum.bin.sto2.STO2Core -s 2 -t 10 -i 1000 -th 12 -a 0.1 -b 0.001/0.1/0.1 -g 1/1 -d asum/in -o asum/out'

In [32]:
#o = subprocess.run(COMMAND, shell=True, capture_output=True)